In [12]:
import torch

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig

from peft import LoraConfig, get_peft_model, TaskType

In [13]:
model_name = 'TinyLLama/TinyLlama-1.1B-Chat-v1.0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [14]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    target_modules = ['q_proj', 'v_proj'],
    lora_dropout = 0.05,
    bias = 'none',
    task_type = TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

In [4]:
#data = load_dataset('csv', data_files='frobinate.csv')['train']
data = load_dataset('json', data_files='frobinate.jsonl')['train']

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
def tokenize(batch):
    texts = [
        f"### Instruction:\n{inst}\n### Response:\n{out}"
        for inst, out in zip(batch['instruction'], batch['response'])
    ]

    tokens = tokenizer(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = 256,
        return_tensors = 'pt'
    )

    tokens['labels'] = tokens['input_ids'].clone()

    return tokens

In [6]:
tokenized_data = data.map(tokenize, batched=True, remove_columns=data.column_names)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [7]:
training_args = TrainingArguments(
    output_dir = './tinyllama-lora-tuned-frobinate',
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    learning_rate = 1e-3,
    num_train_epochs = 50,
    fp16 = True,
    logging_steps = 20,
    save_strategy = 'epoch',
    report_to = 'none',
    remove_unused_columns = False,      
    label_names = ["labels"]
)

In [8]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data,
    processing_class = tokenizer
)

In [9]:
trainer.train()

Step,Training Loss
20,3.130800
40,0.053100
60,0.025200
80,0.024100
100,0.022200
120,0.021300
140,0.020400


TrainOutput(global_step=150, training_loss=0.44099290390809376, metrics={'train_runtime': 171.1902, 'train_samples_per_second': 14.604, 'train_steps_per_second': 0.876, 'total_flos': 2993774886125568.0, 'train_loss': 0.44099290390809376, 'epoch': 37.61538461538461})

In [11]:
model.save_pretrained("./tinyllama-lora-tuned-adapter-frobinate")
tokenizer.save_pretrained("./tinyllama-lora-tuned-adapter-frobinate")

('./tinyllama-lora-tuned-adapter-frobinate/tokenizer_config.json',
 './tinyllama-lora-tuned-adapter-frobinate/special_tokens_map.json',
 './tinyllama-lora-tuned-adapter-frobinate/tokenizer.model',
 './tinyllama-lora-tuned-adapter-frobinate/added_tokens.json',
 './tinyllama-lora-tuned-adapter-frobinate/tokenizer.json')